# Relazione Finale - Gruppo Dig Data
### Alexandru Pavel, Simone Garzarella

# Indice
- [1. Introduzione](#introduction)
    *  [1.1. Descrizione Problema](#problem-description)
    *  [1.2. Specifiche Software](#sw-specs)
    *  [1.3. Specifiche Software](#hw-specs)
- [2. Analisi Dataset](#data-analysis)
    * [2.1. Historical Stock Prices](#hsp)
    * [2.2. Historical Stocks](#hs)
- [3. Job 1](#job1)
    * [3.1. MapReduce](#mapreduce1)
    * [3.2. Hive](#hive1)
    * [3.3. Spark](#spark1)
- [4. Job 2](#job2)
    * [4.1. MapReduce](#mapreduce2)
    * [4.2. Hive](#hive2)
    * [4.3. Spark](#spark2)
- [5. Job 3](#job3)
    * [5.1. MapReduce](#mapreduce3)
    * [5.2. Hive](#hive3)
    * [5.3. Spark](#spark3)
- [6. Grafici Finali](#plots)
    * [6.1. Tipo1](#plot1)
    * [6.2. Tipo2](#plot2)
    * [6.3. Tipo3](#plot3)
- [7. Conclusioni](#conclusions)

## 1. Introduzione <a name="introduction"></a>
Some introduction text, formatted in heading 2 style
### 1.1. Descrizione Problema <a name="problem-description"></a>
This is a sub paragraph, formatted in heading 3 style
### 1.2. Specifiche Software <a name="sw-specs"></a>
This is a sub paragraph, formatted in heading 3 style
### 1.3. Specifiche Hardware <a name="hw-specs"></a>
This is a sub paragraph, formatted in heading 3 style

## 2. Analisi Dataset <a name="data-analysis"></a>
The first paragraph text

### 2.1. Historical Stock Prices <a name="hsp"></a>
This is a sub paragraph, formatted in heading 3 style

### 2.2. Historical Stocks <a name="hs"></a>
This is a sub paragraph, formatted in heading 3 style

## 3. Job 1 <a name="job1"></a>
The first paragraph text

### 3.1. MapReduce <a name="mapreduce1"></a>
This is a sub paragraph, formatted in heading 3 style

### 3.2. Hive <a name="hive1"></a>
This is a sub paragraph, formatted in heading 3 style

### 3.3. Spark <a name="spark1"></a>
This is a sub paragraph, formatted in heading 3 style

## 4. Job 2 <a name="job2"></a>
The first paragraph text

### 4.1. MapReduce <a name="mapreduce2"></a>
This is a sub paragraph, formatted in heading 3 style

### 4.2. Hive <a name="hive2"></a>
This is a sub paragraph, formatted in heading 3 style

### 4.3. Spark <a name="spark2"></a>
This is a sub paragraph, formatted in heading 3 style

## 5. Job 3 <a name="job3"></a>
The first paragraph text

### 5.1. MapReduce <a name="mapreduce3"></a>
This is a sub paragraph, formatted in heading 3 style

### 5.2. Hive <a name="hive3"></a>
This is a sub paragraph, formatted in heading 3 style

### 5.3. Spark <a name="spark3"></a>
This is a sub paragraph, formatted in heading 3 style

## 6. Grafici <a name="plots"></a>
The first paragraph text

### 6.1. MapReduce <a name="plot1"></a>
This is a sub paragraph, formatted in heading 3 style

### 6.2. Hive <a name="plot2"></a>
This is a sub paragraph, formatted in heading 3 style

### 6.3. Spark <a name="plot3"></a>
This is a sub paragraph, formatted in heading 3 style

## 7. Conclusioni <a name="conclusions"></a>
The first paragraph text
